In [1]:
import os
import sys

sys.path.append("../../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from src.utils.helper import Config, color_print
from src.utils.load import load_model, load_data, save_checkpoint
from src.models.evaluate import (
    evaluate_model,
    get_sparsity,
    get_similarity,
    get_perplexity,
)
from src.utils.sampling import SamplingDataset
from src.pruning.prune import prune_magnitude
from src.utils.helper import report_to_df, append_nth_row

In [3]:
name = "bert-6-128-yahoo"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 16
ratio = 0.4
seed = 44
include_layers = ["attention", "intermediate", "output"]
exclude_layers = None

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-10-21 09:24:28


In [5]:
config = Config(name, device)
num_labels = config.config["num_labels"]
model = load_model(config)

Loading the model.

{

'architectures'

: 

'bert'

...

'tokenizer_name'

: 

'fabriceyhc/bert-base-uncased-yahoo_answers_topics'

}

The model models/bert-6-128-yahoo is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    config,
    batch_size=batch_size,
    num_workers=num_workers,
    do_cache=True,
)

Loading cached dataset YahooAnswersTopics.

train.pkl is loaded from cache.

valid.pkl is loaded from cache.

test.pkl is loaded from cache.

The dataset YahooAnswersTopics is loaded

...

,
 

'path'

: 

'yahoo_answers_topics'

}

In [7]:
# print("Evaluate the original model")
# result = evaluate_model(model, config, test_dataloader)

In [8]:
config.init_seed()
all_samples = SamplingDataset(
    train_dataloader,
    config,
    200,
    num_samples,
    False,
    4,
    resample=False,
)

In [9]:
result_list = []

module = copy.deepcopy(model)
prune_magnitude(
    module,
    sparsity_ratio=ratio,
    include_layers=include_layers,
    exclude_layers=exclude_layers,
)
print("Evaluate the pruned model")
result = evaluate_model(model, config, test_dataloader, verbose=True)
result_list.append(result)

Evaluate the pruned model

Evaluating the model:   0%|                                                                               | 0/…

In [10]:
for concern in range(num_labels):
    config.init_seed()
    get_similarity(model, module, valid_dataloader, concern, num_samples, config)

adding eps to diagonal and taking inverse

taking square root

dot products...

trying to take final svd

computed everything!

...

CCA coefficients mean non-concern: 0.9987104570214839

Linear CKA concern: 0.996820492974669

Linear CKA non-concern: 0.9981347197586873

Kernel CKA concern: 0.9912536218568576

Kernel CKA non-concern: 0.9934956450082884

In [11]:
get_sparsity(module)
print("original model's perplexity")
get_perplexity(model, valid_dataloader, config)
print("pruned model's perplexity")
get_perplexity(module, valid_dataloader, config)

0.39177589462304524

{'bert.encoder.layer.0.attention.self.query.weight': 0.39996337890625, 'bert.encoder.layer.0.attention.self.query.bias': 0.0, 'bert.encoder.layer.0.attention.self.key.weight': 0.39996337890625, 'bert.encoder.layer.0.attention.self.key.bias': 0.0, 'bert.encoder.layer.0.attention.self.value.weight': 0.39996337890625, 'bert.encoder.layer.0.attention.self.value.bias': 0.0, 'bert.encoder.layer.0.attention.output.dense.weight': 0.39996337890625, 'bert.encoder.layer.0.attention.output.dense.bias': 0.0, 'bert.encoder.layer.0.intermediate.dense.weight': 0.399993896484375, 'bert.encoder.layer.0.intermediate.dense.bias': 0.0, 'bert.encoder.layer.0.output.dense.weight': 0.399993896484375, 'bert.encoder.layer.0.output.dense.bias': 0.0, 'bert.encoder.layer.1.attention.self.query.weight': 0.39996337890625, 'bert.encoder.layer.1.attention.self.query.bias': 0.0, 'bert.encoder.layer.1.attention.self.key.weight': 0.39996337890625, 'bert.encoder.layer.1.attention.self.key.bias': 0.0, 'bert.encoder.layer.1

original model's perplexity

3.187649726867676

pruned model's perplexity

3.1845717430114746

3.1845717430114746

In [12]:
df_list = [report_to_df(df) for df in result_list]
csv_name = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
new_df = df_list[0]
new_df.to_csv(f"results/{csv_name}.csv", index=False)
print(csv_name)
new_df

2024-10-21_09-25-50

,class,precision,recall,f1-score,support
0,0,0.5581,0.4686,0.5094,2992
1,1,0.7032,0.5194,0.5975,2992
2,2,0.6801,0.6431,0.6611,3012
3,3,0.3396,0.6444,0.4448,2998
4,4,0.7168,0.7635,0.7394,2973
5,5,0.8394,0.7682,0.8022,3054
6,6,0.6735,0.4059,0.5065,3003
7,7,0.6249,0.6062,0.6154,3012
8,8,0.5954,0.6885,0.6386,2982
9,9,0.7322,0.6382,0.6820,2982
